In [15]:
import numpy as np
from pulp import *

In [16]:
# INPUT: 2D vector
z = np.array([[7, 4], [14, 8], [21, 12], [28, 16], [35, 20], [42, 24]])

In [17]:
# INPUT: FUNCTION
def myfunc(x):
    # This uses 2D
    return np.cos(x[:,0])**3+np.tanh(x[:,1])

In [18]:
# Results to replicate
y = myfunc(z)
list(y)

[1.4278236754404783,
 1.0025563598384633,
 0.83567720015193347,
 0.10803972636796511,
 0.26199108387231973,
 0.93600704854681505]

In [19]:
def random_params_to_fit_constraints(X):
    prob = LpProblem("Random Numbers to fit Constraints", LpMinimize)
    # Variables
    # A is in R(d)
    A = {}
    for j in range(X.shape[1]):
        A[j] = LpVariable("a_{}".format(j))
        
    # B is in R(n)
    B = {}
    for i in range(X.shape[0]):
        B[i] = LpVariable("beta_{}".format(i))
        
    # Interleaving Constraints
    # Tolerance for strict inequality
    tolerance = -5
    # b1 < x1 < b2 < x2 < b3 < x3 < b4 < x5
    for i in range(X.shape[0]):
        #b1 < x1, b2 < x2, b3 < x3, b4 < x5
        prob += B[i] <= lpSum( X[i,j] * A[j] for j in range(X.shape[1])) + tolerance
        # x1 < b2, x2 < b3, x3 < b4
        if i < X.shape[0]-1:
            prob += lpSum( X[i,j] * A[j] for j in range(X.shape[1])) <= B[i+1] + tolerance
            
    # Solve
    #print(prob)
    prob.solve()
    #print(LpStatus[prob.status])
    # Extract results
    if prob.status == 1:        
        A_opt = np.zeros(X.shape[1])
        B_opt = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            B_opt[i] = B[i].varValue
        for j in range(X.shape[1]):
            A_opt[j] = A[j].varValue
        return A_opt, [np.dot(X[i], A_opt) for i in range(X.shape[0])], B_opt
    else:
        return None

In [20]:
a, X, b = random_params_to_fit_constraints(z)
#print(a)  # <a,z> = x
print(X)  # Transformed to R1
print(b)  # Bias

[9.9999998000000012, 19.999999600000002, 29.9999994, 39.999999200000005, 49.999999000000003, 59.9999988]
[  0.  15.  25.  35.  45.  55.]


In [21]:
# Constructed nxn matrix
A = np.transpose(np.ones((len(X), len(X)))*X) + np.ones((len(X), len(X)))*-b
A[A<0] = 0
A

array([[  9.9999998,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ],
       [ 19.9999996,   4.9999996,   0.       ,   0.       ,   0.       ,
          0.       ],
       [ 29.9999994,  14.9999994,   4.9999994,   0.       ,   0.       ,
          0.       ],
       [ 39.9999992,  24.9999992,  14.9999992,   4.9999992,   0.       ,
          0.       ],
       [ 49.999999 ,  34.999999 ,  24.999999 ,  14.999999 ,   4.999999 ,
          0.       ],
       [ 59.9999988,  44.9999988,  34.9999988,  24.9999988,  14.9999988,
          4.9999988]])

In [22]:
# Weights matrix
w = np.linalg.solve(A, y) 
w

array([ 0.14278237, -0.37061823,  0.42229592, -0.53444771,  0.71076568,
       -0.60675302])

In [23]:
np.allclose(np.dot(A, w), y)

True

In [24]:
# E.g, y1
w[0]*(X[0]-b[0]), y[0]

(1.4278236754404783, 1.4278236754404783)

In [25]:
# E.g, y2
(w[0]*(X[1]-b[0]))+(w[1]*(X[1]-b[1])), y[1]

(1.0025563598384628, 1.0025563598384633)

In [27]:
# Parameters:
print("2n + d = {}".format(z.shape[0]*2 + z.shape[1]))
print(len(w) + len(b) + len(a))

2n + d = 14
14
